In [20]:
import os
import kagglehub
import pandas as pd
import nltk
import os
import nltk
#nltk.download('punkt_tab')
#nltk.download('stopwords')
print(os.getcwd())
import importlib
import numpy as np
import evaluate
import wandb
from accelerate import Accelerator
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, Dataset
from datasets import Dataset
from transformers import AutoTokenizer, TrainingArguments, Trainer, AutoConfig, AutoModelForSequenceClassification
#nltk.download('punkt_tab')
#nltk.download('stopwords')
from sklearn.model_selection import train_test_split
print(os.getcwd())
from transformers import BertForSequenceClassification
sentences = []
labels = []


'''
with open("amazon_cells_labelled.txt", "r") as file:
    for line in file:
        # Strip whitespace, split the sentence and label
        line = line.strip()
        if line:
            line = line.replace('((25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)(\.|$)){4}', '')
            line = line.replace('[^\w\s]','')
            line = line.replace('\d', '') 
            line = line.replace('\t', '') 
            labels.append(int(line[-1]))
            sentences.append(line[:len(line)-1])
    file.close()
'''


c:\Users\oscar\SKOLA\D7047e\Team-19\Lab 1
c:\Users\oscar\SKOLA\D7047e\Team-19\Lab 1


<>:30: SyntaxWarning: invalid escape sequence '\.'
<>:30: SyntaxWarning: invalid escape sequence '\.'
C:\Users\oscar\AppData\Local\Temp\ipykernel_10940\2845266932.py:30: SyntaxWarning: invalid escape sequence '\.'
  '''


'\nwith open("amazon_cells_labelled.txt", "r") as file:\n    for line in file:\n        # Strip whitespace, split the sentence and label\n        line = line.strip()\n        if line:\n            line = line.replace(\'((25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)(\\.|$)){4}\', \'\')\n            line = line.replace(\'[^\\w\\s]\',\'\')\n            line = line.replace(\'\\d\', \'\') \n            line = line.replace(\'\t\', \'\') \n            labels.append(int(line[-1]))\n            sentences.append(line[:len(line)-1])\n    file.close()\n'

In [21]:
bertbert = "bert-base-uncased"
config = AutoConfig.from_pretrained(bertbert)
tokenizer = AutoTokenizer.from_pretrained(bertbert)
model = AutoModelForSequenceClassification.from_pretrained(bertbert, num_labels=6)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [22]:
"""# Download latest version
path = kagglehub.dataset_download("mohamedbakhet/amazon-books-reviews")

print("Path to dataset files:", path)

#PATH
filepath = '/root/.cache/kagglehub/datasets/mohamedbakhet/amazon-books-reviews/versions/1/Books_rating.csv'
"""

'# Download latest version\npath = kagglehub.dataset_download("mohamedbakhet/amazon-books-reviews")\n\nprint("Path to dataset files:", path)\n\n#PATH\nfilepath = \'/root/.cache/kagglehub/datasets/mohamedbakhet/amazon-books-reviews/versions/1/Books_rating.csv\'\n'

Load dataset, extract only relevant data and split into datasets.

In [23]:
data=pd.read_csv('Books_rating.csv')
print(len(data))
display(data[0:1].T)
data = data[['review/text', 'review/score']].dropna() #Keep only relevant columns and drop missing data.
data['review/score'] = data['review/score'].astype(int)
data = data[data['review/score'].between(0, 5)]
data = data.rename(columns={'review/score': 'labels'})
data=data[0:1000]
#print(data['review/score'][0:100])

train_df, temp_df = train_test_split(data, test_size=0.3, stratify=data['labels'], random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, stratify=temp_df['labels'], random_state=42)

# Convert to Hugging Face Datasets
train_ds = Dataset.from_pandas(train_df.reset_index(drop=True))
val_ds = Dataset.from_pandas(val_df.reset_index(drop=True))
test_ds = Dataset.from_pandas(test_df.reset_index(drop=True))

# Tokenization function
def tokenize_function(batch):
    return tokenizer(batch["review/text"], truncation=True, padding="max_length", max_length=256)

# Apply tokenization
train_ds = train_ds.map(tokenize_function, batched=True)
val_ds = val_ds.map(tokenize_function, batched=True)
test_ds = test_ds.map(tokenize_function, batched=True)


# Set format for PyTorch
cols = ["input_ids", "attention_mask", "labels"]
train_ds.set_format("torch", columns=cols)
val_ds.set_format("torch", columns=cols)
test_ds.set_format("torch", columns=cols)

"""
# Rename label column and convert to 0-indexed
for ds in [train_ds, val_ds, test_ds]:
    ds = ds.rename_column("review/score", "labels")
    ds = ds.map(lambda x: {"labels": x["labels"] - 1})
    ds.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])"""

3000000


,0
Id,1882931173
Title,Its Only Art If Its Well Hung!
Price,NaN
User_id,AVCGYZL8FQQTD
profileName,"Jim of Oz ""jim-of-oz"""
review/helpfulness,7/7
review/score,4.0
review/time,940636800
review/summary,Nice collection of Julie Strain images
review/text,This is only for Julie Strain fans. It's a col...


Map: 100%|██████████| 150/150 [00:00<00:00, 3603.29 examples/s]


'\n# Rename label column and convert to 0-indexed\nfor ds in [train_ds, val_ds, test_ds]:\n    ds = ds.rename_column("review/score", "labels")\n    ds = ds.map(lambda x: {"labels": x["labels"] - 1})\n    ds.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])'

In [24]:
accuracy_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=1)
    return {
        "accuracy": accuracy_metric.compute(predictions=predictions, references=labels)["accuracy"],
        "f1_macro": f1_metric.compute(predictions=predictions, references=labels, average="macro")["f1"],
    }


In [25]:
wandb.login()

wandb.init(project="amazon-books-reviews-sentiment", name="bert-base-6class", reinit=True)


wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: ERROR The nbformat package was not found. It is required to save notebook history.


In [26]:
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="steps",
    logging_steps=50,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    report_to="wandb",  # Log to wandb
    push_to_hub=False,
    fp16=torch.cuda.is_available(),  # Use mixed precision on CUDA
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)


C:\Users\oscar\AppData\Local\Temp\ipykernel_10940\204124960.py:19: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [27]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1 Macro
1,No log,1.122513,0.613333,0.152066
2,1.266500,1.000544,0.613333,0.152066
3,1.025400,0.937041,0.620000,0.164762


TrainOutput(global_step=132, training_loss=1.084872375835072, metrics={'train_runtime': 1416.8885, 'train_samples_per_second': 1.482, 'train_steps_per_second': 0.093, 'total_flos': 276276530073600.0, 'train_loss': 1.084872375835072, 'epoch': 3.0})

In [28]:
trainer.evaluate(test_ds)

{'eval_loss': 1.0047962665557861,
 'eval_accuracy': 0.6133333333333333,
 'eval_f1_macro': 0.15269709543568463,
 'eval_runtime': 26.8216,
 'eval_samples_per_second': 5.593,
 'eval_steps_per_second': 0.373,
 'epoch': 3.0}